Given a "large" DataFrame:
1. save various-sized prefixes as `.parquet`s in S3
2. Convert them to `.duckdb` files (with and without a `UNIQUE INDEX`), also in S3

In [ ]:
from utz import *

[Papermill](https://papermill.readthedocs.io/en/latest/) parameters:

In [ ]:
bkt = 'duckdb-wasm-test'
keys = [ '1e5', '2e5', '5e5', '1e6', '2e6', '4e6', '6e6', ]
full_parquet_path = None  # default: s3://<bkt>/<last key>.parquet, e.g. s3://duckdb-wasm-test/6e6.parquet

In [ ]:
pre = f's3://{bkt}'

In [ ]:
if full_parquet_path is None:
    last = keys[-1]
    full_parquet_path = f'{pre}/{last}.parquet'

## Load full DataFrame

In [ ]:
df = read_parquet(full_parquet_path)
df

In [ ]:
def make_parquet(k):
    """Slice a prefix from `df`, write new Parquet to S3."""
    print(k)
    n = int(float(k))
    d = df.iloc[:n]
    out_path = f'{pre}/{k}.parquet'
    if out_path != full_parquet_path:
        d.to_parquet(out_path) 

## For each "key" in `keys`, write a `.parquet` file to S3

In [ ]:
for k in keys:
    make_parquet(k)

In [ ]:
import boto3
s3 = boto3.client('s3')

In [ ]:
def make_db(idx, index=False):
    """Make a `.duckdb` file from `.parquet` (optionally including a `UNIQUE INDEX` on `id`)."""
    out_name = f'{idx}-idx' if index else f'{idx}'
    db_path = f'{out_name}.duckdb'
    pqt_path = f'{idx}.parquet'
    with duckdb.connect(db_path) as con:
        con.sql(f"CREATE TABLE crashes AS SELECT * FROM '{pre}/{pqt_path}';")
        if index:
            con.sql(f"CREATE UNIQUE INDEX crashes_id_idx ON crashes (id);")
    with open(db_path, 'rb') as f:
        s3.put_object(Bucket=bkt, Key=db_path, Body=f)

In [ ]:
def make_dbs(k):
    """Make two `.duckdb`s from a `.parquet` (one with index, one without)."""
    make_db(k)
    make_db(k, index=True)    

## Make indexed and unindexed `.duckdb`s for each `.parquet`

In [ ]:
for k in keys:
    print(k)
    make_dbs(k)